In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import traceback
import json
import requests
from lxml import html
import time
import tqdm
import plotly_utilities as pu
from IPython import display


In [ ]:
def dt_to_yyyymmdd(d):
    return int(d.year)*100*100 + int(d.month)*100 + int(d.day)

def str_to_yyyymmdd(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    s = '%04d%02d%02d' %(dt.year,dt.month,dt.day)
    return int(s)

def str_to_date(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    return dt


def fetch_history(symbol,dt_beg,dt_end):
    df = yf.download(symbol, dt_beg, dt_end,threads=False)
    # move index to date column, sort and recreate index
    df['date'] = df.index
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df = df.drop(['Adj Close'],axis=1)
    cols = df.columns.values 
    cols_dict = {c:c[0].lower() + c[1:] for c in cols}
    df = df.rename(columns = cols_dict)
    df['settle_date'] = df.date.apply(str_to_yyyymmdd)
    df['symbol'] = symbol
    newcols = ['symbol'] + [c for c in df.columns.values if c != 'symbol']
    return df[newcols].copy()

def get_futures_list_from_yahoo():
    url = 'https://finance.yahoo.com/commodities'
    resp = requests.get(url)
    df_resp = pd.read_html(resp.text)[0]
    return df_resp

def get_yahoo_history_from_list(
    sym_list,
    beg_dt,
    end_dt,
    use_tqdm=False
):
    '''
    Get hisorical prices from yahoo finance for multiple symbols
    @param sym_list: like ['AAPL','CLH22.CME','ES=F']
    @param beg_dt:  like datetime.datetime.now() -  - relativedelta(years=1)
    @param end_dt:  like datetime.datetime.now()
    @use_tqdm:  show progress bar with symbols
    '''
    df_all = pd.DataFrame()
    if use_tqdm:
        _syms = tqdm.tqdm(sym_list,position=0,leave=True)
        _iterator = tqdm.tqdm(sym_list)
    else:
        _syms = sym_list
        _iterator = _syms
    bad_syms = []
    for sym in _iterator:
        if use_tqdm:
            _syms.set_postfix_str(s=sym)    
        try:
            df = fetch_history(sym,beg_dt, end_dt)
            if len(df)>0:
                df_all = df_all.append(df,ignore_index=True)
            else:
                bad_syms.append(sym)
        except:
            bad_syms.append(sym)  
    return df_all,bad_syms

In [ ]:
years_back = 20
df_spx = yf.download('^GSPC', datetime.datetime(1972,1,1), datetime.datetime(2022,10,7),threads=False)
df_spx['close_back'] = df_spx.shift(252*years_back).Close
df_spx2 = df_spx[~df_spx.close_back.isna()]
now_values = df_spx2.Close.values
then_values = df_spx2.close_back.values
date_fraction = 1/(years_back)
df_spx2['perc_incr'] = [
    (now_values[i]/then_values[i])**date_fraction-1
    for i in range(len(now_values))
]

df_spx2['yyyymmdd'] = df_spx2.index.year*100*100 + df_spx2.index.month*100 + df_spx2.index.day
df_spx2['rolling_mean'] = df_spx2.perc_incr.rolling(252).mean()

pu.plotly_plot(
    df_spx2[['yyyymmdd','perc_incr','rolling_mean','Close']],
    x_column='yyyymmdd',
    yaxis2_cols=['Close'],
    plot_title = f' S&P 500 Percent Increase from {years_back} years back, For Each Year',
    y_left_label=f'{years_back} year percent change in Index',
    y_right_label='SP 500 Close'
)


In [ ]:
df_spx3 = df_spx2.copy()
df_spx3['long'] = df_spx3.perc_incr > df_spx3.rolling_mean
df_spx3['prev'] = df_spx3.long.shift(1)
df_spx3['yyyymmdd'] = df_spx3.index.year*100*100 + df_spx3.index.month*100 + df_spx3.index.day
df_spx3.rolling_mean = df_spx3.rolling_mean.fillna(0)

In [ ]:
first_yyyymmdd = df_spx3[df_spx3.rolling_mean==0].yyyymmdd.values[-1]
df_spx4 = df_spx3[['Close','yyyymmdd','long','prev','rolling_mean']][
    (df_spx3.yyyymmdd>=first_yyyymmdd) & (df_spx3.long!=df_spx3.prev)
]
df_spx4.index = range(len(df_spx4))
df_spx4['trade_price'] = df_spx4.Close
df_spx4.loc[df_spx4.long,'trade_price'] = df_spx4.loc[df_spx4.long,'trade_price'] * -1
df_spx4.trade_price = df_spx4.trade_price * -1
# df_spx4.loc[1:len(df_spx4)-2,'trade_price'] = df_spx4.loc[1:len(df_spx4)-2,'trade_price']*2
df_spx4['prev_trade_price'] = df_spx4.trade_price.shift(1)
df_spx4['pl'] = df_spx4.trade_price + df_spx4.prev_trade_price
df_spx4 = df_spx4[~df_spx4.pl.isna()]
df_spx4['cumulative_pl'] = df_spx4.pl.cumsum()


In [ ]:
df_spx4

In [ ]:
pu.plotly_plot(
    df_spx4[['yyyymmdd','cumulative_pl','Close']],
    x_column='yyyymmdd',
    yaxis2_cols=['Close'],
    plot_title = ' S&P 500 Cross Strategy',
    y_left_label='Cummulative P&L',
    y_right_label='SP 500 Close'
)


In [ ]:
def get_perc_increase(years_back,df):
    perc_inc_col  = f'perc_incr_{years_back}'
    df2 = df.copy()
    df2['close_back'] = df2.shift(252*years_back).Close
    df3 = df2[~df2.close_back.isna()].copy()
    now_values = df3.Close.values
    then_values = df3.close_back.values
    date_fraction = 1/(years_back)
    perc_inc_values = [
        (now_values[i]/then_values[i])**date_fraction-1
        for i in range(len(now_values))
    ]
    df3[perc_inc_col] = perc_inc_values
    return df3


In [ ]:
year_arr = [1,5,10,15,20]
dict_df = {
    years:get_perc_increase(years,df_spx)
    for years in year_arr
}

In [ ]:
df_spx5 = dict_df[year_arr[0]].copy()
for year in year_arr[1:]:
    df_spx5 = df_spx5.join(dict_df[year][[f'perc_incr_{year}']])
df_spx5['yyyymmdd']  = df_spx5.index.year*100*100 +  df_spx5.index.month*100 + df_spx5.index.day
df_spx5 = df_spx5[~df_spx5.perc_incr_20.isna()].copy()
df_spx5.index = range(len(df_spx5))

In [ ]:

pu.plotly_plot(
    df_spx5[
        [
            'yyyymmdd',
            'perc_incr_1',
            'perc_incr_5',
            'perc_incr_10',
            'perc_incr_15',
            'perc_incr_20',
            'Close'
        ]
    ],
    x_column='yyyymmdd',
    yaxis2_cols=['Close'],
    plot_title = f' S&P 500 Percent Increase from {",".join([str(y) for y in year_arr])} years back, For Each Year',
    y_left_label=f'year percent change in Index',
    y_right_label='SP 500 Close'
)